# STEP 1 - Installing the requirements

In [2]:
!pip install -r requirements.txt

  Using cached matplotlib-3.3.0.tar.gz (38.8 MB)
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.19.1.zip (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [56 lines of output]
      Running from numpy source directory.
      <string>:470: UserWarning: Unrecognized setuptools command, proceeding with generating Cython sources and expanding templates
      /private/var/folders/w8/s5t40zys2lg2hxnvvxxnyqdm0000gn/T/pip-install-kayv68fr/numpy_946f063222104804a4035bbe92aec78c/tools/cythonize.py:73: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
        required_version = LooseVersion('0.29.21')
      /private/var/folders/w8/s5t40zys2lg2hxnvvxxnyqdm0000gn/T/pip-install-kayv68fr/numpy_946f063222104804a4035bbe92a

---
####**Import necessary libraries**
---

In [3]:
import os
import pandas as pd
import numpy as np
import math
import pickle
import codecs
import argparse
import shutil

#Following code is for debugging

try :
  parser = argparse.ArgumentParser()
  parser.add_argument('--force-retrain', help="Set value to 1 if you wish to retrain the models.")
  parser.add_argument('--debug', help="Debug.")
  args = parser.parse_args()
  force_retrain = args.force_retrain
  debug = args.debug

except :
  force_retrain = None
  debug = None
print ("Done.")

Done.


usage: ipykernel_launcher.py [-h] [--force-retrain FORCE_RETRAIN]
                             [--debug DEBUG]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/narayanaroyal/Library/Jupyter/runtime/kernel-34b109b9-0ed0-47f9-9f0f-32fe4f73e233.json


---
#### **Read Dataset**
---
#### **Dataset Download Link : https://www.kaggle.com/lievgarcia/amazon-reviews**

In [4]:
dataset = pd.read_csv("amazon_dataset_1.csv",encoding='latin1')

len_dataset = dataset.shape[0]

y = dataset[["LABEL"]].values

print ("Done.")

Done.


---
#### **Download nltk Libraries**
---

In [5]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('maxent_treebank_pos_tagger')
nltk.download('averaged_perceptron_tagger')

print("\n---------------------------------------------------------------------------------------\n")

import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

print ("Done.")

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading maxent_treebank_pos_tagger: <urlopen error
[nltk_data]     [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify
[nltk_data]     failed: unable to get local issuer certificate
[nltk_data]     (_ssl.c:997)>



---------------------------------------------------------------------------------------

Done.


[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify
[nltk_data]     failed: unable to get local issuer certificate
[nltk_data]     (_ssl.c:997)>


---
### datacleaning
#### **Tokenization and Stemming**
---


In [6]:
dataset["REVIEW_TEXT"][1]

"Lithium batteries are something new introduced in the market there average developing cost is relatively high but Stallion doesn't compromise on quality and provides us with the best at a low cost.<br />There are so many in built technical assistants that act like a sensor in their particular fort??. The battery keeps my phone charged up and it works at every voltage and a high voltage is never risked."

In [7]:
review = re.sub('[^a-zA-Z]',' ',dataset['REVIEW_TEXT'][1])
review

'Lithium batteries are something new introduced in the market there average developing cost is relatively high but Stallion doesn t compromise on quality and provides us with the best at a low cost  br   There are so many in built technical assistants that act like a sensor in their particular fort    The battery keeps my phone charged up and it works at every voltage and a high voltage is never risked '

In [8]:
review.lower()

'lithium batteries are something new introduced in the market there average developing cost is relatively high but stallion doesn t compromise on quality and provides us with the best at a low cost  br   there are so many in built technical assistants that act like a sensor in their particular fort    the battery keeps my phone charged up and it works at every voltage and a high voltage is never risked '

In [9]:
dataset.head(3)

,DOC_ID,LABEL,RATING,VERIFIED_PURCHASE,PRODUCT_CATEGORY,PRODUCT_ID,PRODUCT_TITLE,REVIEW_TITLE,REVIEW_TEXT
0,1,__label1__,4,N,PC,B00008NG7N,"Targus PAUK10U Ultra Mini USB Keypad, Black",useful,"When least you think so, this product will sav..."
1,2,__label1__,4,Y,Wireless,B00LH0Y3NM,Note 3 Battery : Stalion Strength Replacement ...,New era for batteries,Lithium batteries are something new introduced...
2,3,__label1__,3,N,Baby,B000I5UZ1Q,"Fisher-Price Papasan Cradle Swing, Starlight",doesn't swing very well.,I purchased this swing for my baby. She is 6 m...


In [10]:
dataset["RATING"].unique()

array([4, 3, 1, 2, 5])

In [11]:
dataset["PRODUCT_CATEGORY"].unique()

array(['PC', 'Wireless', 'Baby', 'Office Products', 'Beauty',
       'Health & Personal Care', 'Toys', 'Kitchen', 'Furniture',
       'Electronics', 'Camera', 'Sports', 'Home', 'Jewelry', 'Books',
       'Grocery', 'Video DVD', 'Tools', 'Home Improvement', 'Shoes',
       'Outdoors', 'Luggage', 'Video Games', 'Apparel', 'Lawn and Garden',
       'Pet Products', 'Home Entertainment', 'Musical Instruments',
       'Watches', 'Automotive'], dtype=object)

In [12]:
print ("\nPerforming Tokenization and Stemming.")
load_from_disk = False
corpus=[]
num = 0

for i in range(0, math.floor(len_dataset)) :
        # A progress bar indicates whether the code is completing its job or is stuck in an infinite loop.
    if os.path.exists(os.path.join("Model", "corpus.sav")) and force_retrain == None :
        load_from_disk = True
        continue

    review = re.sub('[^a-zA-Z]',' ',dataset['REVIEW_TEXT'][i])
    review = review.lower()
    review = review.split()
    #print (review)
    review = [word for word in review if not word in set(stopwords.words('english'))]
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)
    

filename = 'corpus.sav'

if load_from_disk :
    corpus = pickle.load(open(os.path.join("Model", filename), 'rb'))
corpus


Performing Tokenization and Stemming.


['least think product save day keep around case need someth',
 'lithium batteri someth new introduc market averag develop cost rel high stallion compromis qualiti provid us best low cost br mani built technic assist act like sensor particular fort batteri keep phone charg work everi voltag high voltag never risk',
 'purchas swing babi month pretti much grown loud swing well beauti though love color lot set think worth money',
 'look inexpens desk calcolatur work everyth need issu tilt slightli one side hit key rock littl bit big deal',
 'use twice week result great use teeth whiten solut result would use least three time week keep use potenc solut also techniqu tray keep everyth teeth mouth',
 'sure suppos would recommend littl research cultur use pipe plan give gift use',
 'pleas ping pong tabl year old year old blast plu lot famili entertain plu better kid sit video game day friend put togeth believ challeng noth could handl',
 'great vitamin c serum realli like oil feel sticki use l

---
#### **Count Vectorization**
---


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=3000)
X=cv.fit_transform(corpus).toarray()
y=dataset.iloc[:len_dataset,1]
print ("Done.")

#filename = 'countvectorizer.sav'
#pickle.dump(cv, open(os.path.join("Model", filename), 'wb'))
X.shape
with open("countvectorizer.sav","wb") as f:
    pickle.dump(cv,f)

Done.


---
#### **POS Tagging**
---


In [23]:
def POS_Tagging(sentence):
    tagged_list = []
    tags = []
    count_verbs = 0
    count_nouns = 0
    text=nltk.word_tokenize(sentence)
    tagged_list = (nltk.pos_tag(text))

    tags = [x[1] for x in tagged_list]
    for each_item in tags:
        if each_item in ['VERB','VB','VBN','VBD','VBZ','VBG','VBP']:
            count_verbs+=1
        elif each_item in ['NOUN','NNP','NN','NUM','NNS','NP','NNPS']:
            count_nouns+=1
        else:
            continue
    if count_verbs > count_nouns:
        sentence = 'F'
    else:
        sentence = 'T'

    return sentence

w, h = 2, len_dataset;
pos_tag = [[0 for x in range(w)] for y in range(h)]
num = 0

load_from_disk = False
filename = 'pos_tag.sav'
print ("\n\nPerforming POS Tagging.")
for i in range(0,len_dataset):
    if os.path.exists(os.path.join("Model", filename)) and force_retrain == None :
        load_from_disk = True
        continue

    text = dataset['REVIEW_TEXT'][i]
    sentence = POS_Tagging(text)

    if sentence == 'T':
        pos_tag[i][0] = 1
        pos_tag[i][1] = 0
        #X[i].insert(1)
        #X[i].insert(0)
    else:
        pos_tag[i][0] = 0
        pos_tag[i][1] = 1

    #print (pos_tag[i])
        #X[i].insert(0)
        #X[i].insert(1)
        
if load_from_disk :
    pos_tag = pickle.load(open(os.path.join("models", filename), 'rb'))

X = np.append(X, pos_tag, axis = 1)
print(X.shape)



Performing POS Tagging.
(21000, 3002)


---
####**Label Encoding**
---

In [24]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

le = LabelEncoder()
y = le.fit_transform(y)

w, h = 3, len_dataset;
new_col = [[0 for x in range(w)] for y in range(h)]
num = 0

for i in range(0, len_dataset):
    new_col[i][0] = dataset["RATING"][i]
    new_col[i][1] = dataset["VERIFIED_PURCHASE"][i]
    new_col[i][2] = dataset["PRODUCT_CATEGORY"][i]

new_col = np.array(new_col)
labelEncoder = LabelEncoder()

new_col[:, 0] = labelEncoder.fit_transform(new_col[:, 0])

new_col[:, 1] = labelEncoder.fit_transform(new_col[:, 1])

new_col[:, 2] = labelEncoder.fit_transform(new_col[:, 2])

print(new_col)
print ("Done.")

[['3' '0' '20']
 ['3' '1' '29']
 ['2' '0' '2']
 ...
 ['4' '1' '22']
 ['4' '1' '22']
 ['3' '1' '22']]
Done.


---
####**OneHotEncoder / Column Transformer**
---

In [25]:
dataset.head(5)

,DOC_ID,LABEL,RATING,VERIFIED_PURCHASE,PRODUCT_CATEGORY,PRODUCT_ID,PRODUCT_TITLE,REVIEW_TITLE,REVIEW_TEXT
0,1,__label1__,4,N,PC,B00008NG7N,"Targus PAUK10U Ultra Mini USB Keypad, Black",useful,"When least you think so, this product will sav..."
1,2,__label1__,4,Y,Wireless,B00LH0Y3NM,Note 3 Battery : Stalion Strength Replacement ...,New era for batteries,Lithium batteries are something new introduced...
2,3,__label1__,3,N,Baby,B000I5UZ1Q,"Fisher-Price Papasan Cradle Swing, Starlight",doesn't swing very well.,I purchased this swing for my baby. She is 6 m...
3,4,__label1__,4,N,Office Products,B003822IRA,Casio MS-80B Standard Function Desktop Calculator,Great computing!,I was looking for an inexpensive desk calcolat...
4,5,__label1__,4,N,Beauty,B00PWSAXAM,Shine Whitening - Zero Peroxide Teeth Whitenin...,Only use twice a week,I only use it twice a week and the results are...


In [26]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
new_col = ohe.fit_transform(new_col)

In [27]:
new_col = new_col.toarray()
X = np.append(X, new_col, axis=1).astype(np.uint8)
X.shape

(21000, 3039)

---
####**Split in Train and Test Set**
---

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

---
####**Training Classifiers**
---

In [29]:
print ("\n\nTraining Classifier on Bernoulli Naive Bayes.")

import pickle 

from sklearn.naive_bayes import BernoulliNB
bernoullinb = BernoulliNB(alpha = 1.0, binarize = 0.0, fit_prior = True, class_prior = None)
bernoullinb.fit(X_train,y_train)
y_pred_bernoulli = bernoullinb.predict(X_test)

from sklearn.metrics import accuracy_score
print ("\nAccuracy of Bernoulli Naive Bayes is : ")
print (accuracy_score(y_test, y_pred_bernoulli) * 100)

with open("bernoullinb_model.pickle","wb") as f:
    pickle.dump(bernoullinb,f)
    
    
    

print ("\n\nTraining Classifier on Support Vector Machine.")
from sklearn.svm import SVC # "Support Vector Classifier"
SVM = SVC(kernel='rbf')
SVM.fit(X_train, y_train)
y_pred_svm = SVM.predict(X_test)

from sklearn.metrics import accuracy_score
print ("\nAccuracy of Support Vector Machine is : ")
print(accuracy_score(y_test, y_pred_svm) * 100)

with open("SVM_model.pickle","wb") as f:
    pickle.dump(SVM,f)



Training Classifier on Bernoulli Naive Bayes.

Accuracy of Bernoulli Naive Bayes is : 
70.16666666666667


Training Classifier on Support Vector Machine.

Accuracy of Support Vector Machine is : 
80.19047619047619
